In [386]:
import pandas as pd
import numpy as np

#import rdkit
#from rdkit import Chem

import biopandas
from biopandas.pdb import PandasPdb
from biopandas.mol2 import PandasMol2

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import sys
from collections import Counter
import time

In [370]:
ANION_MOLECULE = "T2N"
ANION_ATOM_COUNT = 15
ANION_weight = 139.22

CATION_MOLECULE = "BMI"
CATION_ATOM_COUNT = 25
CATION_weight = 281.16

WATER_MOLECULE = "T5P"
WATER_ATOM_COUNT = 3
WATER_weight = 18.02


SODIUM_ATOM = "NA"
CHLORIDE_ATOM = "CL"

SURFACE_RANGE = 5
DIGGER_RANGE = 6

In [371]:
INPUT_FILE = "./FILE/BOX_Bmim_Tf2N_0_5M_all_components_amorphous_-_Frame_51.mol2"
INPUT_FILE_PATH = "./FILE/BOX_Bmim_Tf2N_0_5M_all_components_amorphous_-_Frame_51.mol2"

In [372]:
cnt = 0
find_word = "@<TRIPOS>BOND"
with open(INPUT_FILE, 'r') as file:    # hello.txt 파일을 읽기 모드(r)로 열기
    line = None    # 변수 line을 None으로 초기화
    while line != '':
        line = file.readline()
        cnt += 1
        if "@<TRIPOS>BOND" in line:
            FN = cnt
            
        if "@<TRIPOS>SUBSTRUCTURE" in line:
            EN = cnt
#===============================================================================           
f = open(INPUT_FILE_PATH, 'r')
F = f.readlines()[FN:EN-1]
Bond_list = []
for i in range(EN-FN-1):
    DATA = F[int(i)].strip().split()
    Bond_list.append(DATA)
#===============================================================================  
column_name=['Bond_Index', 'Bond_Atom1', 'Bond_Atom2', 'Bond_Case']
Bond_Data_Frame = pd.DataFrame(Bond_list, columns = column_name)
#===============================================================================
Bond_Data_Frame = Bond_Data_Frame.astype({'Bond_Atom1': int, 'Bond_Atom2': int})

In [373]:
pmol = PandasMol2().read_mol2(INPUT_FILE_PATH)  ### input
Total_system = pmol.df

In [374]:
condition = (pmol.df.subst_name == CATION_MOLECULE) 
Data_Selecte_subst_CATION = pmol.df[condition]
Data_Selecte_subst_CATION_index = pmol.df[condition].index

condition = (pmol.df.subst_name == ANION_MOLECULE) 
Data_Selecte_subst_ANION = pmol.df[condition]
Data_Selecte_subst_ANION_index = pmol.df[condition].index

condition = (pmol.df.subst_name == WATER_MOLECULE) 
Data_Selecte_subst_WATER = pmol.df[condition]
Data_Selecte_subst_WATER_index = pmol.df[condition].index

condition = (pmol.df.subst_name == SODIUM_ATOM) 
Data_Selecte_subst_NA = pmol.df[condition]
Data_Selecte_subst_NA_index = pmol.df[condition].index


condition = (pmol.df.subst_name == CHLORIDE_ATOM) 
Data_Selecte_subst_CL = pmol.df[condition]
Data_Selecte_subst_CL_index = pmol.df[condition].index

In [375]:
CATION_Residue_LIST = []
for i in range(1, int(len(Data_Selecte_subst_CATION_index)/CATION_ATOM_COUNT)+1):
    ATOM_COUNT = 0  ## while 반복문 초기화
    while ATOM_COUNT < 25:    ## 수치 적정 변수로 생각하기
        CATION_Residue_LIST.append("CATION_Res_"+str(i))
        ATOM_COUNT = ATOM_COUNT+1
        
        
ANION_Residue_LIST = []
for i in range(1, int(len(Data_Selecte_subst_ANION_index)/ANION_ATOM_COUNT)+1):
    ATOM_COUNT = 0  ## while 반복문 초기화
    while ATOM_COUNT < 15:    ## 수치 적정 변수로 생각하기
        ANION_Residue_LIST.append("ANION_Res_"+str(i))
        ATOM_COUNT = ATOM_COUNT+1       
        
        
WATER_Residue_LIST = []
for i in range(1, int(len(Data_Selecte_subst_WATER_index)/WATER_ATOM_COUNT)+1):
    ATOM_COUNT = 0  ## while 반복문 초기화
    while ATOM_COUNT < 3:    ## 수치 적정 변수로 생각하기
        WATER_Residue_LIST.append("WATER_Res_"+str(i))
        ATOM_COUNT = ATOM_COUNT+1        
        
        
NA_Residue_LIST = []
for i in range(1, int(len(Data_Selecte_subst_NA_index)/1)+1):
    ATOM_COUNT = 0  ## while 반복문 초기화
    while ATOM_COUNT < 1:    ## 수치 적정 변수로 생각하기
        NA_Residue_LIST.append("NA_Res_"+str(i))
        ATOM_COUNT = ATOM_COUNT+1        
        
        
CL_Residue_LIST = []
for i in range(1, int(len(Data_Selecte_subst_CL_index)/1)+1):
    ATOM_COUNT = 0  ## while 반복문 초기화
    while ATOM_COUNT < 1:    ## 수치 적정 변수로 생각하기
        CL_Residue_LIST.append("CL_Res_"+str(i))
        ATOM_COUNT = ATOM_COUNT+1        

In [376]:
SYSTEM_Reside_LIST = CATION_Residue_LIST + ANION_Residue_LIST + WATER_Residue_LIST + NA_Residue_LIST + CL_Residue_LIST

In [377]:
SYSTEM_Reside_DF = pd.DataFrame(SYSTEM_Reside_LIST)
SYSTEM_Reside_DF.columns=["Residue_number"]

In [378]:
Total_system_SYSTEM_Reside_DF = pd.concat([Total_system,SYSTEM_Reside_DF],axis=1)

In [379]:
### 1. 특정 ATOM을 선택하면 해당 index를 뽑고 해당 Residue_number를 추출한다.
### 2. 해당 Residue_number에 해당하는 분자를 찾는다.

In [380]:
len(Total_system_SYSTEM_Reside_DF)

42660

In [381]:
len(Total_system_SYSTEM_Reside_DF[(Total_system_SYSTEM_Reside_DF['subst_name'] == CATION_MOLECULE)])

9675

## Define Cluster

#### step 1. 선택한 molecule의 인근 (3A) ATOM 선택하기

In [382]:
condition = (pmol.df.subst_name == ANION_MOLECULE)
Data_Selecte_subst = pmol.df[condition]
Data_Selecte_subst_index = pmol.df[condition].index

In [389]:
start = time.time()

find_range = 3    ### input
list_set = []
list_set_array = np.array([], dtype= int)


for i in range(0, len(Data_Selecte_subst_index)):
    select_dataframe = None  ## 선택한 데이터 프레임 초기화
    select_dataframe = Data_Selecte_subst.iloc[i]


    sq1_range = (select_dataframe[['x','y','z']]+find_range)
    sq3_range = (select_dataframe[['x','y','z']]-find_range)

    selected_dataframe = Total_system_SYSTEM_Reside_DF[(Total_system_SYSTEM_Reside_DF['x'] <= sq1_range["x"]) &
                                                       (Total_system_SYSTEM_Reside_DF['x'] >= sq3_range["x"]) &
                                                       (Total_system_SYSTEM_Reside_DF['y'] <= sq1_range["y"]) &
                                                       (Total_system_SYSTEM_Reside_DF['y'] >= sq3_range["y"]) &
                                                       (Total_system_SYSTEM_Reside_DF['z'] <= sq1_range["z"]) &
                                                       (Total_system_SYSTEM_Reside_DF['z'] >= sq3_range["z"])]
                                      

    cal_x = (float(select_dataframe["x"]) - selected_dataframe["x"]).pow(2)
    cal_y = (float(select_dataframe["y"]) - selected_dataframe["y"]).pow(2)
    cal_z = (float(select_dataframe["z"]) - selected_dataframe["z"]).pow(2)
    Distance = (cal_x+cal_y+cal_z)**0.5
    
    #Distance

    Distance_index = Distance[(Distance <= find_range)].index
    Select_Atom_List = Distance_index.values.tolist()
    Select_Atom_List_NP = np.array(Select_Atom_List,dtype=int)
    
    list_set_array = np.append(list_set_array,Select_Atom_List_NP)
list_set_array= np.unique(list_set_array)
    
    ##  선택한 원자 선택
    
    # list_set
    # list_set = [] 위쪽에 빈공간 있음
    #for i in range(0,len(Select_Atom_List)+1):
    #    list_set.extend(Select_Atom_List)
    #    list_set = set(list_set)  
    #    list_set = list(list_set)
        
        
print("time :",time.time() - start)

time : 16.69072723388672


In [390]:
list_set_array

array([    5,    10,    11, ..., 42489, 42526, 42530])

In [391]:
list_set

[]

In [ ]:
start = time.time()

find_range = 3    ### input
list_set = []
list_set_array = np.array([], dtype= int)


for i in range(0, len(Data_Selecte_subst_index)):
    select_dataframe = None  ## 선택한 데이터 프레임 초기화
    select_dataframe = Data_Selecte_subst.iloc[i]


    sq1_range = (select_dataframe[['x','y','z']]+find_range)
    sq3_range = (select_dataframe[['x','y','z']]-find_range)

    selected_dataframe = Total_system_SYSTEM_Reside_DF[(Total_system_SYSTEM_Reside_DF['x'] <= sq1_range["x"]) &
                                                       (Total_system_SYSTEM_Reside_DF['x'] >= sq3_range["x"]) &
                                                       (Total_system_SYSTEM_Reside_DF['y'] <= sq1_range["y"]) &
                                                       (Total_system_SYSTEM_Reside_DF['y'] >= sq3_range["y"]) &
                                                       (Total_system_SYSTEM_Reside_DF['z'] <= sq1_range["z"]) &
                                                       (Total_system_SYSTEM_Reside_DF['z'] >= sq3_range["z"])]
                                      

    cal_x = (float(select_dataframe["x"]) - selected_dataframe["x"]).pow(2)
    cal_y = (float(select_dataframe["y"]) - selected_dataframe["y"]).pow(2)
    cal_z = (float(select_dataframe["z"]) - selected_dataframe["z"]).pow(2)
    Distance = (cal_x+cal_y+cal_z)**0.5
    
    #Distance

    Distance_index = Distance[(Distance <= find_range)].index
    Select_Atom_List = Distance_index.values.tolist()
    Select_Atom_List_NP = np.array(Select_Atom_List,dtype=int)
    
    list_set_array = np.append(list_set_array,Select_Atom_List_NP)
list_set_array= np.unique(list_set_array)
    
    ##  선택한 원자 선택
    
    # list_set
    # list_set = [] 위쪽에 빈공간 있음
    #for i in range(0,len(Select_Atom_List)+1):
    #    list_set.extend(Select_Atom_List)
    #    list_set = set(list_set)  
    #    list_set = list(list_set)
        
        
print("time :",time.time() - start)

#### step 2. 선택한 molecule에서 Cation Residue만 선택하기

In [237]:
# 포함하고자 하는 문자열 리스트 생성
Select_Residue_List = Total_system_SYSTEM_Reside_DF.iloc[list_set][["Residue_number"]].drop_duplicates().squeeze().to_list()

# 해당 list_set 있는 잔기를 선택
result = Total_system_SYSTEM_Reside_DF[Total_system_SYSTEM_Reside_DF['Residue_number'].isin(Select_Residue_List)]

In [238]:
result[result['subst_name'] == CATION_MOLECULE]

,atom_id,atom_name,x,y,z,atom_type,subst_id,subst_name,charge,Residue_number
0,1,C,21.7125,12.9756,-21.7713,C.3,1,BMI,-0.3305,CATION_Res_1
1,2,N,22.9275,12.3355,-21.4824,N.pl3,1,BMI,0.1939,CATION_Res_1
2,3,C,22.9824,11.0176,-21.2151,C.ar,1,BMI,-0.1044,CATION_Res_1
3,4,N,24.2717,10.6997,-20.8984,N.pl3,1,BMI,0.2896,CATION_Res_1
4,5,C,25.0390,11.8696,-20.9131,C.ar,1,BMI,-0.3225,CATION_Res_1
...,...,...,...,...,...,...,...,...,...,...
9670,9671,H,22.3638,-11.8768,4.0557,H,1,BMI,0.0914,CATION_Res_387
9671,9672,H,22.3671,-13.5961,4.2719,H,1,BMI,0.0882,CATION_Res_387
9672,9673,HXT,24.2591,-11.2821,9.7294,H,1,BMI,0.2511,CATION_Res_387
9673,9674,HXT,27.8447,-11.9993,7.5262,H,1,BMI,0.2562,CATION_Res_387


#### step 3. 선택한 molecule의 인근 (3A) ATOM 선택하기

In [239]:
Data_Selecte_subst = result[result['subst_name'] == CATION_MOLECULE]

In [240]:
Data_Selecte_subst_index = result[result['subst_name'] == CATION_MOLECULE].index

In [241]:

find_range = 3    ### input
list_set = []

for i in range(0, len(Data_Selecte_subst_index)):
    select_dataframe = None  ## 선택한 데이터 프레임 초기화
    select_dataframe = Data_Selecte_subst.iloc[i]


    sq1_range = (select_dataframe[['x','y','z']]+find_range)
    sq3_range = (select_dataframe[['x','y','z']]-find_range)

    selected_dataframe = Total_system[(Total_system['x'] <= sq1_range["x"]) &
                                      (Total_system['x'] >= sq3_range["x"]) &
                                      (Total_system['y'] <= sq1_range["y"]) &
                                      (Total_system['y'] >= sq3_range["y"]) &
                                      (Total_system['z'] <= sq1_range["z"]) &
                                      (Total_system['z'] >= sq3_range["z"])]
                                      

    cal_x = (float(select_dataframe["x"]) - selected_dataframe["x"]).pow(2)
    cal_y = (float(select_dataframe["y"]) - selected_dataframe["y"]).pow(2)
    cal_z = (float(select_dataframe["z"]) - selected_dataframe["z"]).pow(2)
    Distance = (cal_x+cal_y+cal_z)**0.5
    
    #Distance

    Distance_index = Distance[(Distance <= find_range)].index
    Select_Atom_List=Distance_index.values.tolist()
    
    ##  선택한 원자 선택
    
    # list_set
    # list_set = [] 위쪽에 빈공간 있음
    for i in range(0,len(Select_Atom_List)+1):
        list_set.extend(Select_Atom_List)
        list_set = set(list_set)  
        list_set = list(list_set)


In [242]:
# 포함하고자 하는 문자열 리스트 생성
Select_Residue_List = Total_system_SYSTEM_Reside_DF.iloc[list_set][["Residue_number"]].drop_duplicates().squeeze().to_list()

# 해당 list_set 있는 잔기를 선택
result = Total_system_SYSTEM_Reside_DF[Total_system_SYSTEM_Reside_DF['Residue_number'].isin(Select_Residue_List)]

#### step 4. IL Cluster Define

In [243]:
IL_Cluster = result[(result['subst_name'] == CATION_MOLECULE)|(result['subst_name'] == ANION_MOLECULE)]

In [346]:
IL_Cluster

,atom_id,atom_name,x,y,z,atom_type,subst_id,subst_name,charge,Residue_number
0,1,C,21.7125,12.9756,-21.7713,C.3,1,BMI,-0.3305,CATION_Res_1
1,2,N,22.9275,12.3355,-21.4824,N.pl3,1,BMI,0.1939,CATION_Res_1
2,3,C,22.9824,11.0176,-21.2151,C.ar,1,BMI,-0.1044,CATION_Res_1
3,4,N,24.2717,10.6997,-20.8984,N.pl3,1,BMI,0.2896,CATION_Res_1
4,5,C,25.0390,11.8696,-20.9131,C.ar,1,BMI,-0.3225,CATION_Res_1
...,...,...,...,...,...,...,...,...,...,...
15475,15476,F,18.5276,30.0993,-18.6566,F,2,T2N,-0.1518,ANION_Res_387
15476,15477,O,17.4059,33.6747,-17.9482,O.2,2,T2N,-0.6559,ANION_Res_387
15477,15478,O,16.6167,31.5197,-16.9102,O.2,2,T2N,-0.6559,ANION_Res_387
15478,15479,O,15.8857,34.0225,-20.4904,O.2,2,T2N,-0.6559,ANION_Res_387


## Select 5A Surface

#### step 1. Select 5A surface

In [255]:
Data_Selecte_subst = IL_Cluster
Data_Selecte_subst_index = IL_Cluster.index

In [256]:

find_range = SURFACE_RANGE    ### input
list_set = []

for i in range(0, len(Data_Selecte_subst_index)):
    select_dataframe = None  ## 선택한 데이터 프레임 초기화
    select_dataframe = Data_Selecte_subst.iloc[i]


    sq1_range = (select_dataframe[['x','y','z']]+find_range)
    sq3_range = (select_dataframe[['x','y','z']]-find_range)

    selected_dataframe = Total_system[(Total_system['x'] <= sq1_range["x"]) &
                                      (Total_system['x'] >= sq3_range["x"]) &
                                      (Total_system['y'] <= sq1_range["y"]) &
                                      (Total_system['y'] >= sq3_range["y"]) &
                                      (Total_system['z'] <= sq1_range["z"]) &
                                      (Total_system['z'] >= sq3_range["z"])]
                                      

    cal_x = (float(select_dataframe["x"]) - selected_dataframe["x"]).pow(2)
    cal_y = (float(select_dataframe["y"]) - selected_dataframe["y"]).pow(2)
    cal_z = (float(select_dataframe["z"]) - selected_dataframe["z"]).pow(2)
    Distance = (cal_x+cal_y+cal_z)**0.5
    
    #Distance

    Distance_index = Distance[(Distance <= find_range)].index
    Select_Atom_List=Distance_index.values.tolist()
    
    ##  선택한 원자 선택
    
    # list_set
    # list_set = [] 위쪽에 빈공간 있음
    for i in range(0,len(Select_Atom_List)+1):
        list_set.extend(Select_Atom_List)
        list_set = set(list_set)  
        list_set = list(list_set)


In [257]:
# 포함하고자 하는 문자열 리스트 생성
Select_Residue_List = Total_system_SYSTEM_Reside_DF.iloc[list_set][["Residue_number"]].drop_duplicates().squeeze().to_list()

# 해당 list_set 있는 잔기를 선택
result = Total_system_SYSTEM_Reside_DF[Total_system_SYSTEM_Reside_DF['Residue_number'].isin(Select_Residue_List)]

In [258]:
IL_Cluster_surface = result[(result['subst_name'] == CATION_MOLECULE)|
                            (result['subst_name'] == ANION_MOLECULE)|
                            (result['subst_name'] == WATER_MOLECULE)|
                            (result['subst_name'] == SODIUM_ATOM)|
                            (result['subst_name'] == CHLORIDE_ATOM)]

In [263]:
len(Total_system_SYSTEM_Reside_DF[(Total_system_SYSTEM_Reside_DF['subst_name'] == CATION_MOLECULE)])

9675

In [264]:
len(IL_Cluster_surface[(IL_Cluster_surface['subst_name'] == CATION_MOLECULE)])

9650

#### step 2. Invert

In [295]:
IL_Cluster_surface_NAME = IL_Cluster_surface["Residue_number"].drop_duplicates().squeeze().to_list()

In [297]:
IL_Cluster_surface_invert = Total_system_SYSTEM_Reside_DF[~Total_system_SYSTEM_Reside_DF['Residue_number'].isin(IL_Cluster_surface_NAME)]

#### step 3. Select 5A surface From Invert moleuce

In [302]:
Data_Selecte_subst = IL_Cluster_surface_invert
Data_Selecte_subst_index = IL_Cluster_surface_invert.index

In [304]:

find_range = DIGGER_RANGE    ### input
list_set = []

for i in range(0, len(Data_Selecte_subst_index)):
    select_dataframe = None  ## 선택한 데이터 프레임 초기화
    select_dataframe = Data_Selecte_subst.iloc[i]


    sq1_range = (select_dataframe[['x','y','z']]+find_range)
    sq3_range = (select_dataframe[['x','y','z']]-find_range)

    selected_dataframe = Total_system[(Total_system['x'] <= sq1_range["x"]) &
                                      (Total_system['x'] >= sq3_range["x"]) &
                                      (Total_system['y'] <= sq1_range["y"]) &
                                      (Total_system['y'] >= sq3_range["y"]) &
                                      (Total_system['z'] <= sq1_range["z"]) &
                                      (Total_system['z'] >= sq3_range["z"])]
                                      

    cal_x = (float(select_dataframe["x"]) - selected_dataframe["x"]).pow(2)
    cal_y = (float(select_dataframe["y"]) - selected_dataframe["y"]).pow(2)
    cal_z = (float(select_dataframe["z"]) - selected_dataframe["z"]).pow(2)
    Distance = (cal_x+cal_y+cal_z)**0.5
    
    #Distance

    Distance_index = Distance[(Distance <= find_range)].index
    Select_Atom_List=Distance_index.values.tolist()
    
    ##  선택한 원자 선택
    
    # list_set
    # list_set = [] 위쪽에 빈공간 있음
    for i in range(0,len(Select_Atom_List)+1):
        list_set.extend(Select_Atom_List)
        list_set = set(list_set)  
        list_set = list(list_set)


In [306]:
# 포함하고자 하는 문자열 리스트 생성
Select_Residue_List = Total_system_SYSTEM_Reside_DF.iloc[list_set][["Residue_number"]].drop_duplicates().squeeze().to_list()

# 해당 list_set 있는 잔기를 선택
result = Total_system_SYSTEM_Reside_DF[Total_system_SYSTEM_Reside_DF['Residue_number'].isin(Select_Residue_List)]

In [307]:
IL_Cluster_digger = result[(result['subst_name'] == CATION_MOLECULE)|
                           (result['subst_name'] == ANION_MOLECULE)|
                           (result['subst_name'] == WATER_MOLECULE)|
                           (result['subst_name'] == SODIUM_ATOM)|
                           (result['subst_name'] == CHLORIDE_ATOM)]

#### step 4. Define inside Water using invert

In [309]:
IL_Cluster_digger_NAME = IL_Cluster_digger["Residue_number"].drop_duplicates().squeeze().to_list()

In [310]:
IL_Cluster_digger_INSIDE = Total_system_SYSTEM_Reside_DF[~Total_system_SYSTEM_Reside_DF['Residue_number'].isin(IL_Cluster_digger_NAME)]

In [311]:
INSIDE_water = IL_Cluster_digger_INSIDE[(IL_Cluster_digger_INSIDE['subst_name'] == WATER_MOLECULE)]

In [319]:
len(INSIDE_water)/3

440.0

## ANALYSIS

In [325]:
import sys
from collections import Counter

In [357]:
# (CMC) CATION_MOLECULE_COUNT
CMC = Counter(IL_Cluster['subst_name'])[CATION_MOLECULE] / CATION_ATOM_COUNT

In [358]:
# (AMC) ANION_MOLECULE_COUNT
AMC = Counter(IL_Cluster['subst_name'])[ANION_MOLECULE] / ANION_ATOM_COUNT

In [359]:
# (WMC) WATER_MOLECULE_COUNT
WMC = Counter(INSIDE_water['subst_name'])['T5P'] / WATER_ATOM_COUNT

In [363]:
Absortion = (WMC*WATER_weight)/((CMC*CATION_weight)+(AMC*ANION_weight)+(WMC*WATER_weight))*100

In [364]:
print(CMC,AMC,WMC,Absortion)

386.0 385.0 440.0 4.662457000994847
